In [79]:
import json
import math 
import os
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from word2vec import get_vector

from eden_ai import get_labels

In [80]:
no_of_bins = 20

videos = {}
with open('videos.json', 'r') as f: 
    videos = json.load(f)

x = []
y = []

for video in videos.values():
    description = video["video"]["description"].split() or []
    tags = video["video"].get('tags', []) or []
    labels = [inner["label"] for inner in video["video"]["thumbnail_objects"]] or []

    words = description + tags + labels
    x.append(get_vector(words))

    view = video['video']['view_count']
    sub_count = video['channel']['subscriber_count']

    y.append(int(math.tanh(math.log(view) / math.log(sub_count)) * no_of_bins))

In [81]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [82]:
# Create an instance of the Random Forest Regression model
model = RandomForestRegressor()

# Train the model on the training data
model.fit(x_train, y_train)

RandomForestRegressor()

In [83]:
# Make predictions on the test data
y_pred = model.predict(x_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

for index in range(len(x_test)):
    print(index, y_pred[index], y_test[index])

Mean Squared Error: 0.6769685393258427
0 13.03 11
1 12.69 13
2 13.48 13
3 12.94 13
4 13.58 14
5 12.98 14
6 13.05 13
7 12.73 14
8 12.96 14
9 13.04 12
10 13.13 13
11 13.4 13
12 13.34 14
13 13.6 14
14 12.86 14
15 13.16 13
16 12.97 12
17 13.19 13
18 13.63 14
19 13.62 14
20 13.27 13
21 13.06 14
22 12.65 12
23 13.11 12
24 13.38 13
25 12.32 12
26 12.92 13
27 13.12 13
28 13.3 12
29 13.42 14
30 12.89 14
31 13.7 14
32 13.05 14
33 12.94 12
34 12.75 12
35 13.27 13
36 13.37 14
37 13.25 14
38 12.99 14
39 13.16 12
40 12.63 13
41 13.08 13
42 12.83 13
43 13.09 13
44 13.01 14
45 13.67 14
46 13.5 14
47 13.15 14
48 12.75 14
49 12.87 13
50 13.16 13
51 13.14 15
52 13.36 13
53 12.48 13
54 13.23 13
55 13.43 14
56 12.73 13
57 12.91 12
58 13.05 14
59 13.27 14
60 12.89 13
61 12.57 12
62 13.34 12
63 13.64 13
64 13.23 13
65 12.48 13
66 13.21 13
67 13.18 13
68 12.71 13
69 12.84 14
70 13.16 14
71 12.89 13
72 13.41 14
73 13.12 14
74 12.61 14
75 12.96 13
76 13.26 14
77 13.08 13
78 12.92 12
79 12.73 14
80 12.98 10
81 1

In [84]:
def predict(tags, description, thumbnail_url):
    labels = [inner['label'] for inner in get_labels(image_url=thumbnail_url)] or []

    words = description + tags + labels
    words_vector = get_vector(words)

    return model.predict([words_vector])[0]

In [85]:
tags = ["duzce", "duzce earthquake", "duzce news", "duzce turkey", "earthquake", 
               "earthquake in turkey", "erdogan", "gdnpfpnewsworld", "golkaya", "golkaya earthquake", 
               "haber", "istanbul", "istanbul earthquake", "istanbul news", "istanbul news anchor", 
               "tgrt", "turkey", "turkey 2022", "turkey earthquake", "turkey earthquake live", "turkey earthquake video", 
               "turkey latest", "turkey latest news", "turkey news", "turkey quake", "turkish anchor", "turkish news reporter", 
               "turkish reporter", "world"]

description = """A magnitude-5.9 earthquake hit a town in northwestern Turkey early Wednesday, causing damage to some buildings and widespread panic. 
Subscribe to Guardian News on YouTube ► http://bit.ly/guardianwiressub

At least 68 people were injured, mostly while trying to flee homes. Footage released by Turkish private broadcaster TGRT shows how night-time anchorman Ersel Hoskara coninued presenting as the quake was felt in Istanbul while the graphic underneath him read 'Breaking news: moment of the earthquake live on air'. The earthquake was centered in the town of Golkaya, in Duzce province, some 200 kilometers (125 miles) east of Istanbul, the Disaster and Emergency Management Presidency said. 

The Guardian publishes independent journalism, made possible by supporters. Contribute to The Guardian today ► https://bit.ly/3uhA7zg

Sign up to the Guardian's free new daily newsletter, First Edition ► http://theguardian.com/first-edition

Website ► https://www.theguardian.com
Facebook ►https://www.facebook.com/theguardian
Twitter ► https://twitter.com/guardian
Instagram ► https://instagram.com/guardian

The Guardian on YouTube:
The Guardian ► https://bit.ly/guardiannewssubs
Guardian Australia ► https://bit.ly/guardianaussubs
Guardian Football ► https://bit.ly/gdnfootballsubs
Guardian Sport ► https://bit.ly/gdnsportsubs
Guardian Live ► https://bit.ly/guardianlivesubs

#Istanbul #News #Earthquake #Istanbul #News""".split()

thumbnail_url = "https://i3.ytimg.com/vi/SP9_USUZn68/maxresdefault.jpg"

y = predict(tags, description, thumbnail_url)
print(y)

13.13
